# Chapter 7: Ensemble Learning dan Random Forests

Bab ini membahas pendekatan *Ensemble Learning*, yaitu teknik Machine Learning yang menggabungkan beberapa model untuk meningkatkan akurasi dan stabilitas prediksi. Pendekatan ini banyak digunakan dalam sistem produksi dan kompetisi Machine Learning karena mampu mengurangi kesalahan generalisasi.

Alih-alih mengandalkan satu model tunggal, Ensemble Learning memanfaatkan kekuatan kolektif dari beberapa model yang memiliki karakteristik berbeda.


## 1. Konsep Dasar Ensemble Learning

Ensemble Learning didasarkan pada prinsip bahwa sekumpulan model yang beragam sering kali mampu menghasilkan prediksi yang lebih baik dibandingkan satu model tunggal.

Dalam Ensemble Learning:
- Model individual disebut *base learner*
- Gabungan beberapa model disebut *ensemble*
- Prediksi akhir diperoleh melalui agregasi hasil prediksi


### 1.1 Weak Learner dan Strong Learner

Weak learner adalah model dengan performa sedikit lebih baik dari tebakan acak. Meskipun lemah secara individual, kombinasi banyak weak learner dapat menghasilkan strong learner yang sangat akurat.

Keberhasilan pendekatan ini sangat bergantung pada keberagaman kesalahan antar model.


## 2. Voting Classifier

Voting Classifier merupakan bentuk Ensemble Learning paling sederhana. Beberapa model dilatih secara terpisah, kemudian prediksinya digabungkan menggunakan mekanisme pemungutan suara.


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

log_reg = LogisticRegression()
rf_clf = RandomForestClassifier()
svm_clf = SVC()

voting_hard = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf_clf),
        ('svm', svm_clf)
    ],
    voting='hard'
)


### 2.1 Evaluasi Voting Classifier

Untuk mengevaluasi performa Voting Classifier, digunakan dataset sintetis dan metrik akurasi.


In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = make_moons(n_samples=10000, noise=0.35, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

for model in (log_reg, rf_clf, svm_clf, voting_hard):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.__class__.__name__, accuracy_score(y_test, y_pred))


### 2.2 Soft Voting

Soft voting menggunakan probabilitas prediksi dari setiap model. Probabilitas tersebut dirata-ratakan, dan kelas dengan probabilitas tertinggi dipilih sebagai hasil akhir.


In [ ]:
svm_clf = SVC(probability=True)

voting_soft = VotingClassifier(
    estimators=[
        ('lr', log_reg),
        ('rf', rf_clf),
        ('svm', svm_clf)
    ],
    voting='soft'
)


## 3. Bagging dan Pasting

Bagging dan Pasting merupakan teknik Ensemble Learning yang meningkatkan diversitas model dengan melatih setiap model pada subset data yang berbeda.


### 3.1 Bagging (Bootstrap Aggregating)

Bagging menggunakan teknik sampling dengan pengembalian (bootstrap). Akibatnya, satu data dapat muncul lebih dari satu kali dalam satu subset.

Pendekatan ini efektif untuk mengurangi variance, terutama pada model yang tidak stabil seperti Decision Tree.


### 3.2 Pasting

Pasting menggunakan sampling tanpa pengembalian. Setiap subset data berisi instance yang unik.

Dibandingkan Bagging, Pasting cenderung menghasilkan model dengan tingkat keberagaman yang lebih rendah.


### 3.3 Bagging with Scikit-Learn

Scikit-Learn menyediakan class `BaggingClassifier` untuk mengimplementasikan teknik Bagging secara langsung.


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)

bag_clf.fit(X_train, y_train)


### 3.4 Training and Evaluating Bagging Classifier

Setelah model dilatih, performanya dievaluasi menggunakan data test untuk melihat peningkatan akurasi dibandingkan Decision Tree tunggal.


In [ ]:
y_pred_bag = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred_bag)


## 4. Out-of-Bag (OOB) Evaluation

Out-of-Bag evaluation memungkinkan estimasi performa model tanpa validation set terpisah, dengan memanfaatkan data yang tidak terpilih selama proses bootstrap sampling.


### 4.1 Apa itu Out-of-Bag Samples?

Pada setiap bootstrap sampling, sekitar 37% data tidak terpilih. Data ini disebut out-of-bag samples dan dapat digunakan untuk evaluasi model.


### 4.2 Mengaktifkan OOB Evaluation di Scikit-Learn

OOB evaluation dapat diaktifkan dengan mengatur parameter `oob_score=True`.


In [ ]:
bag_oob = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42
)

bag_oob.fit(X_train, y_train)
bag_oob.oob_score_


## 5. Random Forest

Random Forest merupakan ensemble Decision Tree yang dilatih menggunakan Bagging dengan tambahan randomisasi fitur pada setiap split.


### 5.1 Mengapa Random Forest Bekerja dengan Baik?

Decision Tree tunggal cenderung mengalami overfitting karena sangat sensitif terhadap variasi data.
Random Forest mengatasi masalah ini dengan dua mekanisme utama:

1. **Bootstrap sampling**: setiap pohon dilatih pada subset data yang berbeda.
2. **Random feature selection**: setiap split hanya mempertimbangkan sebagian fitur secara acak.

Kombinasi ini menghasilkan pohon-pohon yang tidak saling berkorelasi kuat, sehingga prediksi gabungannya menjadi lebih stabil dan akurat.


### 5.2 Random Forest for Classification

Pada tugas klasifikasi, Random Forest menggabungkan prediksi dari setiap pohon menggunakan mekanisme majority voting.
Kelas yang paling sering diprediksi oleh seluruh pohon akan dipilih sebagai hasil akhir.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

rf_clf.fit(X_train, y_train)


### 5.3 Evaluating the Random Forest

Setelah model Random Forest dilatih, performanya dievaluasi menggunakan data test untuk mengukur kemampuan generalisasi model.


In [ ]:
from sklearn.metrics import accuracy_score

y_pred_rf = rf_clf.predict(X_test)
accuracy_score(y_test, y_pred_rf)


### 5.4 Random Forest vs Bagging

Random Forest pada dasarnya adalah Bagging Decision Tree dengan tambahan randomisasi fitur.
Perbedaan utamanya adalah:

- **Bagging**: menggunakan seluruh fitur pada setiap split.
- **Random Forest**: hanya menggunakan subset fitur secara acak.

Randomisasi fitur ini meningkatkan diversitas antar pohon dan umumnya menghasilkan performa yang lebih baik.


## 6. Boosting

Boosting melatih model secara berurutan, di mana setiap model baru berfokus memperbaiki kesalahan model sebelumnya.


### 6.1 Perbedaan Extra-Trees dan Random Forest

Perbedaan utama antara kedua metode ini adalah:

- Random Forest mencari threshold terbaik pada subset fitur.
- Extra-Trees memilih threshold secara acak tanpa optimasi.

Akibatnya, Extra-Trees:
- lebih cepat dilatih,
- memiliki bias sedikit lebih tinggi,
- tetapi variansi yang lebih rendah.


### 6.2 Training an Extra-Trees Classifier

Scikit-Learn menyediakan class `ExtraTreesClassifier` untuk mengimplementasikan metode ini.


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

extra_clf = ExtraTreesClassifier(
    n_estimators=500,
    max_leaf_nodes=16,
    n_jobs=-1,
    random_state=42
)

extra_clf.fit(X_train, y_train)


### 6.3 Evaluating Extra-Trees

Performa Extra-Trees dievaluasi menggunakan data test untuk dibandingkan dengan Random Forest.


In [ ]:
y_pred_extra = extra_clf.predict(X_test)
accuracy_score(y_test, y_pred_extra)


### 6.4 Feature Importance

Random Forest dan Extra-Trees mampu mengestimasi tingkat kepentingan setiap fitur berdasarkan kontribusinya dalam mengurangi impurity.
Informasi ini sangat berguna untuk interpretasi model dan feature selection.


In [ ]:
rf_clf.feature_importances_


## 7. Boosting

Boosting merupakan teknik ensemble yang melatih model secara berurutan.
Setiap model baru difokuskan untuk memperbaiki kesalahan yang dibuat oleh model sebelumnya.


### 7.1 Intuisi Boosting

Boosting bekerja dengan memberikan perhatian lebih besar pada data yang sulit diprediksi.
Dengan pendekatan ini, sekumpulan weak learner dapat dikombinasikan menjadi model yang sangat kuat.


### 7.2 AdaBoost (Adaptive Boosting)

AdaBoost merupakan algoritma boosting yang menyesuaikan bobot data secara adaptif.
Instance yang salah diprediksi akan diberi bobot lebih besar pada iterasi berikutnya.


### 7.2.1 Training an AdaBoost Classifier

Secara default, AdaBoost menggunakan Decision Tree dengan kedalaman satu sebagai weak learner.


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)

ada_clf.fit(X_train, y_train)


### 7.2.2 Evaluating AdaBoost

Model AdaBoost dievaluasi menggunakan data test untuk mengukur performanya.


In [ ]:
y_pred_ada = ada_clf.predict(X_test)
accuracy_score(y_test, y_pred_ada)


### 7.3 Gradient Boosting

Gradient Boosting melatih model baru untuk memprediksi residual (kesalahan) dari model sebelumnya.
Pendekatan ini mirip dengan optimasi menggunakan gradient descent.


### 7.3.1 Intuisi Gradient Boosting

Model dilatih secara bertahap, di mana setiap model berusaha meminimalkan kesalahan model sebelumnya.
Learning rate digunakan untuk mengontrol kontribusi setiap model baru.


### 7.3.2 Training a Gradient Boosting Regressor

Contoh berikut menunjukkan penerapan Gradient Boosting untuk masalah regresi.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

gbrt.fit(X_train, y_train)


## 8. Stacking

Stacking merupakan teknik ensemble lanjutan yang menggunakan model tingkat kedua (meta-learner)
untuk mempelajari cara terbaik menggabungkan prediksi dari beberapa model dasar.


### 8.1 Intuisi Stacking

Prediksi dari model-model dasar digunakan sebagai fitur input bagi meta-learner.
Pendekatan ini memungkinkan ensemble belajar bagaimana menimbang kontribusi setiap model.


### 8.2 Stacking in Practice

Dalam praktik, meta-learner biasanya menggunakan model sederhana seperti Logistic Regression
untuk menjaga kemampuan generalisasi.


## Closing Summary (Chapter 7)

Bab ini membahas berbagai teknik Ensemble Learning, termasuk Voting, Bagging, Random Forest, Boosting, dan Stacking. Pendekatan ini menunjukkan bahwa kombinasi beberapa model yang beragam mampu menghasilkan sistem Machine Learning yang lebih kuat, stabil, dan akurat dibandingkan model tunggal.
